In [1]:
# imports
from tkinter import ttk
import tkinter as tk
from PIL import ImageTk, Image
from io import BytesIO
import requests
import json
import csv
import pandas as pd

In [ ]:
# functions to call in the GUI (add these into the gui)

# search button command (might use ingredient input function instead)
def ingredient():


# add button command (we might have to get rid of this part)
def add_ingredient():
    ingredient_list = []



def user_input():
    ingredient_list = []
    while True:
        users_input = input("Do you want to add an ingredient to your list? y/n: ").lower()
        if users_input == "y":
            ingredient = input("Add an ingredient: ").lower()
            ingredient_list.append(ingredient)
        elif users_input == "n":
            break
        else:
            print("Please enter 'y' or 'n'!")

# backend function to add csvs
def search_recipes(ingredient):
    csv_file = open("recipes.csv", "w", newline="", encoding="utf-8")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["Recipe Name", "Thumbnail Link", "Ingredients", "Cook Time (in minutes)", "Diet", "Cuisine", "Recipe Source"])
    
    key = "5cb09004fe27449689868405b5a33789"
    find_ingredients = requests.get(f"https://api.spoonacular.com/recipes/findByIngredients?ingredients={ingredient}&apiKey={key}")
    ingredients_json = find_ingredients.json()
    
    for recipe in ingredients_json:
        id_num = recipe["id"]
        recipe_info = requests.get(f"https://api.spoonacular.com/recipes/{id_num}/information?apiKey={key}").json()
        recipe_name = recipe_info["title"]
        recipe_pic = recipe_info["image"]
        cook_time = recipe_info["readyInMinutes"]
        recipe_diets = recipe_info["diets"]
        if recipe_info["cuisines"]:
            recipe_cuisine = recipe_info["cuisines"]
        else:
            recipe_cuisine = "NaN"
        ingredients_list = []
        for i in recipe_info["extendedIngredients"]:
            ingredient_name = i["nameClean"]
            ingredients_list.append(ingredient_name)
        recipe_source = recipe_info["sourceUrl"]
        csv_writer.writerow([recipe_name, recipe_pic, ingredients_list, cook_time, recipe_diets, recipe_cuisine, recipe_source])
    csv_file.close()

    data = pd.read_csv("recipes.csv")
    recipe_name_list = data["Recipe Name"].tolist()
    img_list = data["Thumbnail Link"].tolist()
    for i in range(len(recipe_name_list)): # add this into the gui
        recipe_box = tk.Frame(recipes, bg="gray", bd=2, padx=10, pady=10)
        recipe_box.pack(side="left", padx=10, pady=10)
        recipe_title = tk.Label(recipe_box, text=recipe_name_list[i], bg="gray")
        recipe_title.pack(anchor="w")
        recipe_response = requests.get(img_list[i])
        info = BytesIO(recipe_response)
        recipe_img = Image.open(info)
        recipe_img.thumbnail((200, 200))
        recipe_pic = ImageTk.PhotoImage(recipe_img)
        recipe_panel = tk.Label(recipe_box, image=recipe_pic, bg="light gray")
        recipe_panel.image = recipe_pic
        recipe_panel.pack(side="center", padx=10)
    

#search_recipes(we would add the .get from the search bar here)